In [ ]:
!pip install openai openai-agents dotenv

Definir modelo y conexión
-----

In [ ]:
from openai import AsyncAzureOpenAI
from agents import set_default_openai_client
from dotenv import load_dotenv
import os

# Set environment variables
os.environ["AZURE_OPENAI_API_KEY"] = "your-key"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com/"
os.environ["AZURE_OPENAI_DEPLOYMENT"] = "gpt-4o"

# Load environment variables
load_dotenv()

# Create OpenAI client using Azure OpenAI
client = AsyncAzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT")
)

# Set the default OpenAI client for the Agents SDK
set_default_openai_client(client)

Crear Agente
-----

In [ ]:
MENU_PRICES = """🍰 Dulces Delicias - Menú de Pastelería 🍰
🎂 Tortas y Pasteles
Torta de Chocolate Clásica – $4.50 por porción | $28.00 entera
Cheesecake – $5.00 por porción | $30.00 entera
Torta Red Velvet – $4.75 por porción | $29.00 entera
Tiramisú Italiano – $5.50 por porción | $32.00 entera
Torta Tres Leches – $4.25 por porción | $27.00 entera
🧁 Cupcakes y Muffins
Cupcake de Vainilla con Buttercream – $2.50 c/u
Cupcake de Chocolate y Avellanas – $2.75 c/u
Muffin de Arándanos – $2.80 c/u
Muffin de Banana y Nueces – $2.90 c/u
🥐 Panadería Artesanal
Croissant de Mantequilla – $3.00
Pain au Chocolat – $3.50
Roll de Canela con Glaseado – $3.75
Empanada de Manzana – $4.00
🍪 Galletas y Snacks
Galletas de Chispas de Chocolate – $1.50 c/u | $8.00 por 6 unidades
Macarons Franceses (varios sabores) – $2.20 c/u | $12.00 por caja de 6
Brownie de Chocolate Intenso – $3.50 c/u
Alfajor de Dulce de Leche – $2.75 c/u
🍹 Bebidas
Café Espresso – $2.50
Café Latte – $3.50
Capuchino – $3.75
Chocolate Caliente – $3.25
Té Natural (varios sabores) – $2.00
Limonada Natural – $3.00
Malteada de Vainilla/Chocolate/Fresa – $4.50"""

In [ ]:
from agents import Agent, Runner, OpenAIChatCompletionsModel


agent = Agent(name="Asistente de pasteleria",
            instructions=f"Eres un camarero de una pasteleria que da información de precios de este menu: \n {MENU_PRICES}",
            model= OpenAIChatCompletionsModel(
                                model="gpt-4o",
                                openai_client=client
                  )
            )

In [ ]:
import nest_asyncio
nest_asyncio.apply()

Ejecución síncrona

In [ ]:
result = Runner.run_sync(agent, "Cuanto cuesta el capuchino?.")
print(result.final_output)

In [ ]:
result = Runner.run_sync(agent, "Cuanto cuestan 3 porciones de cheesecake?.")
print(result.final_output)

Ejecución Asíncrona

In [ ]:
result = await Runner.run(agent, "Cuanto cuesta el capuchino?.")
print(result.final_output)

In [ ]:
result = await Runner.run(agent, "Cuanto cuestan 3 porciones de cheesecake?.")
print(result.final_output)

Ejecución streaming mode

In [ ]:
import asyncio
from openai.types.responses import ResponseTextDeltaEvent

result = Runner.run_streamed(agent, input="cuanto cuesta cada plato del menu?")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

In [ ]:
result = Runner.run_streamed(agent, input="cuanto cuesta 2 porciones de cada plato del menu?")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)